In [2]:
from collections import namedtuple

SequenceRecordInformation = namedtuple("SequenceRecordInformation", ["ForwardSequence", "ReverseSequence", "ForwardSequenceLength", "ReverseSequenceLength", "Id", "Description"])

In [3]:
#imports 
from Bio.Seq import Seq
from Bio import SeqIO
from Bio import Align
from Bio.SeqRecord import SeqRecord
from Bio.Align.Applications import ClustalwCommandline
from Bio import AlignIO

#For loop for turning the .fa files into usable variables and returning the lengths of the sequences. 
NS_H9 = list(SeqIO.parse("db/NS/sequences/NS_H9_sequences.fa", "fasta"))

sequence_record_information = []

for sequence in NS_H9: 
        NS_H9_text = sequence.seq.upper()
        NS_H9_sequence = Seq(NS_H9_text)
        # print(len(NS_H9_sequence))
        forward_sequence = NS_H9_sequence[0:200]
        reverse_sequence = NS_H9_sequence[-200:]
        sr = SequenceRecordInformation(forward_sequence, reverse_sequence, len(forward_sequence), len(reverse_sequence), sequence.id, sequence.description)
        sequence_record_information.append(sr)
        

SeqIo.parse takes the .fa files and turns them into usable variables that we can then maniplualte. The first two lines of the for loop turn the now usable .fa file into a sequence object where all the text is in uppercase. It then prints the length of each of the sequences includes in the .fa file. The fourth and fifth lines of the for loop slice the sequences so that only the first and last 200 basepairs are included and saves these edited sequences as usable variables (forward and reverse sequence respectably). It then prints the edited sequences and the lengths of the sequences to ensure that they the correct sequences and are only 200 basepairs long. 

In [4]:
for sequences in sequence_record_information:
        print(f'Sequence ID: {sequences.Id}')
        print(f'Sequence Description: {sequences.Description}')
        print(f'Forward Sequence: {sequences.ForwardSequence}')
        print(f'Reverse Sequence: {sequences.ReverseSequence}')
        print(f'Forward Sequence Length: {sequences.ForwardSequenceLength}')
        print(f'Reverse Sequence Length: {sequences.ReverseSequenceLength}') 
        print()

Sequence ID: MF440740
Sequence Description: MF440740 A/Beijing/1/2017 2017/04/28 8 (NS)
Forward Sequence: ATGGATTCCAATACTGTGTCAAGCTTCCAGGTAGACTGCTTTCTTTGGCATGTCCGCAAACGATTTGCAGACCAAGAAATGGGTGATGCCCCATTTCTAGACCGGCTTCGCCGAGATCAGAAGTCCCTGAGAGGAAGAAGCAGCACTCTTGGTCTGGACATCAGAACCGCCACGCATGAAGGAAAGCATATAGTGGAGCG
Reverse Sequence: TGGAGAGAACAGTTAAGCCAGAAATTCGAAGAAATAAGATGGTTGATTGAAGAAGTACGACATAGATTAAAAATTACGGAGAATAGCTTTGAGCAAATAACTTTTATGCAAGCCTTACAACTATTGCTTGAAGTGGAGCAAGAGATAAGAACTTTCTCGTTTCAGCTTATTTAATGATAAAAAACACCCTTGTTTCTACT
Forward Sequence Length: 200
Reverse Sequence Length: 200

Sequence ID: MK673900
Sequence Description: MK673900 A/India/TCM2581/2019 2019/02/02 8 (NS)
Forward Sequence: AGCAGGGTGACAAAAACATAATGGATTCCAACACTGTGTCAAGCTTTCAGGTAGATTGCTTTCTTTGGCATGTCCGCAAACGATTTGCAGACCAAGAACTGTGTGATGCCCCATTCCTTGACCGGCTTCGCCGAGATCAGAAGTCCTTAAGAGGAAGGAGCAGCACTCTTGGTCTGAATATCGAGACAGCTACTCGTGCA
Reverse Sequence: CAGAAATGGGAAATGGCGAGAACAATTAAGCCAGAAGTTTGGGGAGATAAGATGGCTAATTGAAGAAGTGCGACATAGACTGAAAA

In [5]:
#Turning edited sequences into sequence record object to be used in alignments 

forward_sequence_records = []
reverse_sequence_records = []

for record_information in sequence_record_information: 
    forward_record = SeqRecord(record_information.ForwardSequence, id = record_information.Id, description = record_information.Description)
    reverse_record = SeqRecord(record_information.ReverseSequence, id = record_information.Id, description= record_information.Description)
    forward_sequence_records.append(forward_record)
    reverse_sequence_records.append(reverse_record)

 

In [6]:
#Making the primers into sequence record objects, and saving them as .fasta files  
NS_forward_primer= SeqRecord(Seq("GCAAAAGCAGGGTGACAAA"), id= "NS Forward 1.2", description= "Forward primer")
NS_reverse_primer= Seq("GATCAGTAGAAACAAGGGTGTT")
NS_reverse_compliment = Seq(NS_reverse_primer.reverse_complement())
NS_revcomp_primer= SeqRecord(Seq(NS_reverse_compliment), id= "NS Reverse 1.1", description= "Reverse primer reverse compliment")

SeqIO.write(NS_forward_primer, "NS_forward_primer.fasta", "fasta")
SeqIO.write(NS_revcomp_primer, "NS_revcomp_primer.fasta", "fasta")

1

In [7]:
#List of sequences to be aligned
H9_NS_edited_forward = forward_sequence_records + [NS_forward_primer]
H9_NS_edited_reverse = reverse_sequence_records + [NS_revcomp_primer]



#Saving list of sequences to be aligned as .fasta files 
SeqIO.write(H9_NS_edited_forward, "H9_NS_edited_forward.fasta", "fasta")
SeqIO.write(H9_NS_edited_reverse, "H9_NS_edited_reverse.fasta", "fasta")

4

In [8]:
###ClustalW alignment 
#Forward sequences alignment
cline = ClustalwCommandline("clustalw2", infile='H9_NS_edited_forward.fasta')
print(cline)

stdout, stderr = cline()

align = AlignIO.read("H9_NS_edited_forward.aln", "clustal")
print(align)


#Reverse sequences alignment 
cline = ClustalwCommandline("clustalw2", infile='H9_NS_edited_reverse.fasta')
print(cline)

stdout, stderr = cline()

align = AlignIO.read("H9_NS_edited_reverse.aln", "clustal")
print(align)


clustalw2 -infile=H9_NS_edited_forward.fasta
Alignment with 4 rows and 220 columns
--------------------ATGGATTCCAATACTGTGTCAAGC...GCG MF440740
--------------------ATGGATTCCAATACTGTGTCAAGC...GCG MT875141
AGCAGGGTGACAAAAACATAATGGATTCCAACACTGTGTCAAGC...--- MK673900
--------------------------------------------...--- NS
clustalw2 -infile=H9_NS_edited_reverse.fasta
Alignment with 4 rows and 226 columns
-------------CAGAAATGGGAAATGGCGAGAACAATTAAGC...--- MK673900
ACTCTCTACAAAGTAGAAACGGGAAATGGAGAGAACAGTTAAGC...--- MT875141
--------------------------TGGAGAGAACAGTTAAGC...ACT MF440740
--------------------------------------------...--- NS
